In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from optbinning import OptimalBinning, Scorecard, BinningProcess
import import_ipynb
from function_for_eda import *

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


(CVXPY) Dec 12 02:41:13 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Dec 12 02:41:13 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
importing Jupyter notebook from function_for_eda.ipynb


# 1. APPLICATION_TRAIN.CSV AND APPLICATION_TEST.CSV
Các bảng application_train và application_test chứa các thông tin cá nhân trực tiếp liên quan đến những người đi vay vì vậy đây là bảng chính và quan trọng nhất. Từ các bước EDA trước đó, chúng ta tiến hành các bước Feature Engineering như sau (một điều lưu ý rằng tất cả các bước khi thực hiện ở bảng application_train thì đều phải thực hiện tương tự ở bảng application_test):
- <b>Bước 1</b>: Xóa các cột không liên quan 'Unnamed: 0', xử lý các giá trị bất thường bằng cách thay thế nó bằng Nan
- <b>Bước 2</b>: Thực hiện các bước data processing cơ bản như xóa đi các cột chứa missing value trên 60%, fill các giá trị nan, chuyển các cột chứa giá trị ngày thành năm, xóa các cột chỉ chứa 1 giá trị, xử lý các giá trị outliers
- <b>Bước 3</b>: Tạo một số feature mới dựa trên Domain Knowledge, qua trình EDA và một số solutions tham khảo khác
- <b>Bước 4</b>: Sử dụng BinningProcess từ thư viện optbinning để tìm cách phân nhóm tối ưu cho một biến nhất định (ở đây là các biến categorical), đảm bảo rằng các nhóm có ý nghĩa thống kê và mang lại sự phân tách tốt nhất giữa các lớp.
- <b>Bước 5 </b>: Encoding các biến categorical

### APPLICATION

In [2]:
application_train= pd.read_csv('../dseb63_final_project_DP_dataset/dseb63_application_train.csv')
application_train

,Unnamed: 0,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SK_ID_CURR
0,0,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,278621
1,1,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,139008
2,2,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,138348
3,3,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,64140
4,4,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,219374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246004,246004,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,90253
246005,246005,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,9027
246006,246006,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0,163598
246007,246007,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,300375


In [3]:
application_test = pd.read_csv('../dseb63_final_project_DP_dataset/dseb63_application_test.csv')
application_test

,Unnamed: 0,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SK_ID_CURR
0,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,...,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0,83659
1,1,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,...,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0,174814
2,2,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,...,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0,179486
3,3,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,57038
4,4,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,234000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0,25672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61497,61497,Cash loans,M,Y,Y,0,171000.0,521280.0,23089.5,450000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0,150442
61498,61498,Revolving loans,M,Y,Y,2,450000.0,900000.0,45000.0,900000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,5217
61499,61499,Cash loans,F,N,Y,0,225000.0,202500.0,24030.0,202500.0,...,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0,260741
61500,61500,Cash loans,M,N,Y,0,121500.0,254700.0,30357.0,225000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,284794


In [4]:
# Drop column 'Unnamed: 0'
application_train.drop('Unnamed: 0', axis = 1, inplace = True)

application_test.drop('Unnamed: 0', axis = 1, inplace = True)

In [5]:
## Anomolies
application_train['DAYS_EMPLOYED'][application_train['DAYS_EMPLOYED'] == 365243] = np.nan
application_test['DAYS_EMPLOYED'][application_test['DAYS_EMPLOYED'] == 365243] = np.nan

application_train['OBS_30_CNT_SOCIAL_CIRCLE'][application_train['OBS_30_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_train['OBS_60_CNT_SOCIAL_CIRCLE'][application_train['OBS_60_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_test['OBS_30_CNT_SOCIAL_CIRCLE'][application_test['OBS_30_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_test['OBS_60_CNT_SOCIAL_CIRCLE'][application_test['OBS_60_CNT_SOCIAL_CIRCLE'] > 30] == np.nan


Series([], Name: OBS_60_CNT_SOCIAL_CIRCLE, dtype: bool)

In [6]:
application_train = remove_missing_col(application_train, 66)
application_test = remove_missing_col(application_test, 66.3)

application_train = fill_nan(application_train)
application_test = fill_nan(application_test)

day_cols = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']
years_cols = ['AGE', 'YEARS_EMPLOYED', 'YEARS_REGISTRATION', 'YEARS_ID_PUBLISH' ]

application_train = create_day_to_year(application_train,day_cols, years_cols)
application_test = create_day_to_year(application_test,day_cols, years_cols)

application_train = drop_column_unique_value(application_train)
application_test = drop_column_unique_value(application_test)

application_train = replace_outlier(application_train)
application_test = replace_outlier(application_test)




## Create Feature

In [7]:
## Numeric feature Engineering
def numeric_feature_engineer(data):
    
    #income and credit features
    data['CREDIT_INCOME_RATIO'] = data['AMT_CREDIT'] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    data['CREDIT_ANNUITY_RATIO'] = data['AMT_CREDIT'] / (data['AMT_ANNUITY'] + 0.00001)
    data['ANNUITY_INCOME_RATIO'] = data['AMT_ANNUITY'] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    data['INCOME_ANNUITY_DIFF'] = data['AMT_INCOME_TOTAL'] - data['AMT_ANNUITY']
    data['CREDIT_GOODS_RATIO'] = data['AMT_CREDIT'] / (data['AMT_GOODS_PRICE'] + 0.00001)
    data['CREDIT_GOODS_DIFF'] = data['AMT_CREDIT'] - data['AMT_GOODS_PRICE'] + 0.00001
    data['GOODS_INCOME_RATIO'] = data['AMT_GOODS_PRICE'] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    data['PAYMENT_RATE'] = data['AMT_ANNUITY'] / (data['AMT_CREDIT'] + 0.00001)
    data['INCOME_CREDIT_PERC'] =  data['AMT_INCOME_TOTAL'] / (data['AMT_CREDIT'] + 0.00001)
    data['INCOME_TO_EMPLOYED_RATIO'] = data['AMT_INCOME_TOTAL'] / (data['YEARS_EMPLOYED'] + 0.00001)


    #age ratios and diffs
    data['AGE_EMPLOYED_DIFF'] = data['AGE'] - data['YEARS_EMPLOYED']
    data['EMPLOYED_TO_AGE_RATIO'] = data['YEARS_EMPLOYED'] / (data['AGE'] + 0.00001)
    data['INCOME_TO_BIRTH_RATIO'] = data['AMT_INCOME_TOTAL'] / (data['AGE'] + 0.00001)      
    data['ID_TO_BIRTH_RATIO'] = data['YEARS_ID_PUBLISH'] / (data['AGE'] + 0.00001)
  
    #car ratios
    data['CAR_EMPLOYED_DIFF'] = data['OWN_CAR_AGE'] - data['YEARS_EMPLOYED']
    data['CAR_EMPLOYED_RATIO'] = data['OWN_CAR_AGE'] / (data['YEARS_EMPLOYED']+0.00001)
    data['CAR_AGE_DIFF'] = data['AGE'] - data['OWN_CAR_AGE']
    data['CAR_AGE_RATIO'] = data['OWN_CAR_AGE'] / (data['AGE'] + 0.00001)
    
    #flag contacts sum
    data['FLAG_CONTACTS_SUM'] =  data['FLAG_EMP_PHONE'] + data['FLAG_WORK_PHONE'] +  data['FLAG_PHONE'] + data['FLAG_EMAIL']

    #family members
    data['CNT_NON_CHILDREN'] = data['CNT_FAM_MEMBERS'] - data['CNT_CHILDREN']
    data['PER_CAPITA_INCOME'] = data['AMT_INCOME_TOTAL'] / (data['CNT_FAM_MEMBERS'] + 1)
    data['CHILDREN_RATIO'] =  data['CNT_CHILDREN'] / (data['CNT_FAM_MEMBERS'] + 1)


    #region ratings
    data['REGIONS_RATING_INCOME_MUL'] = (data['REGION_RATING_CLIENT'] + data['REGION_RATING_CLIENT_W_CITY']) * data['AMT_INCOME_TOTAL'] / 2
    data['REGION_RATING_MAX'] = [max(ele1, ele2) for ele1, ele2 in zip(data['REGION_RATING_CLIENT'], data['REGION_RATING_CLIENT_W_CITY'])]
    data['REGION_RATING_MAX'] = [min(ele1, ele2) for ele1, ele2 in zip(data['REGION_RATING_CLIENT'], data['REGION_RATING_CLIENT_W_CITY'])]
    data['REGION_RATING_MEAN'] = (data['REGION_RATING_CLIENT'] + data['REGION_RATING_CLIENT_W_CITY']) / 2
    data['REGION_RATING_MUL'] = data['REGION_RATING_CLIENT'] * data['REGION_RATING_CLIENT_W_CITY']
    
    #flag regions
    data['FLAG_REGIONS'] =  data['REG_REGION_NOT_WORK_REGION'] + data['LIVE_REGION_NOT_WORK_REGION'] + data['REG_CITY_NOT_LIVE_CITY'] + \
                            data['REG_CITY_NOT_WORK_CITY'] + data['LIVE_CITY_NOT_WORK_CITY']
    
    #ext_sources
    data['EXT_SOURCE_MEAN'] = (data['EXT_SOURCE_1'] + data['EXT_SOURCE_2'] + data['EXT_SOURCE_3'] ) / 3
    data['EXT_SOURCE_MUL'] = data['EXT_SOURCE_1'] * data['EXT_SOURCE_2'] * data['EXT_SOURCE_3'] 
    data['EXT_SOURCE_MAX'] = [max(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
    data['EXT_SOURCE_MIN'] = [min(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
    data['EXT_SOURCE_VAR'] = [np.var([ele1,ele2,ele3]) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
    data['WEIGHTED_EXT_SOURCE'] =  data['EXT_SOURCE_1'] * 2 + data['EXT_SOURCE_2'] * 3 + data['EXT_SOURCE_3'] * 4
    
    #OBS And DEF
    data['OBS_30_60_SUM'] = data['OBS_30_CNT_SOCIAL_CIRCLE'] + data['OBS_60_CNT_SOCIAL_CIRCLE']
    data['DEF_30_60_SUM'] = data['DEF_30_CNT_SOCIAL_CIRCLE'] + data['DEF_60_CNT_SOCIAL_CIRCLE']
    data['OBS_DEF_30_MUL'] = data['OBS_30_CNT_SOCIAL_CIRCLE'] *  data['DEF_30_CNT_SOCIAL_CIRCLE']
    data['OBS_DEF_60_MUL'] = data['OBS_60_CNT_SOCIAL_CIRCLE'] *  data['DEF_60_CNT_SOCIAL_CIRCLE']
    data['SUM_OBS_DEF_ALL'] = data['OBS_30_CNT_SOCIAL_CIRCLE'] + data['DEF_30_CNT_SOCIAL_CIRCLE'] + data[
        'OBS_60_CNT_SOCIAL_CIRCLE'] + data['DEF_60_CNT_SOCIAL_CIRCLE']
    data['OBS_30_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['OBS_30_CNT_SOCIAL_CIRCLE'] + 0.00001)
    data['OBS_60_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['OBS_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
    data['DEF_30_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['DEF_30_CNT_SOCIAL_CIRCLE'] + 0.00001)
    data['DEF_60_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['DEF_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
    
    #Flag Documents combined
    data['SUM_FLAGS_DOCUMENTS'] = data['FLAG_DOCUMENT_3'] + data['FLAG_DOCUMENT_6'] + data['FLAG_DOCUMENT_8']         

    #enquires
    
    data['AMT_ENQ_SUM'] = data['AMT_REQ_CREDIT_BUREAU_HOUR'] + data['AMT_REQ_CREDIT_BUREAU_DAY'] + data['AMT_REQ_CREDIT_BUREAU_WEEK'] + data[
        'AMT_REQ_CREDIT_BUREAU_MON'] + data['AMT_REQ_CREDIT_BUREAU_QRT'] + data['AMT_REQ_CREDIT_BUREAU_YEAR']
    data['ENQ_CREDIT_RATIO'] = data['AMT_ENQ_SUM'] / (data['AMT_CREDIT'] + 0.00001)
    
 
    return data

In [8]:
application_train = numeric_feature_engineer(application_train)
application_test = numeric_feature_engineer(application_test)

In [9]:
print('Training Features shape: ', application_train.shape)
print('Testing Features shape: ', application_test.shape)

Training Features shape:  (246009, 146)
Testing Features shape:  (61502, 145)


In [10]:
# binning process
target_var = application_train['TARGET']
application_train.drop('TARGET', axis = 1, inplace = True)
cate_cols = application_train.select_dtypes(include='object').columns.tolist()

columns = application_train.columns.tolist()
binning_process = BinningProcess(columns, categorical_variables=cate_cols, max_n_prebins=30)

binning_process.fit(application_train, target_var)

# Transform train and test
train_binned = binning_process.transform(application_train, metric_missing=0.05)
train_binned.columns = [f'{col}_BIN' for col in train_binned.columns]
train_binned['SK_ID_CURR'] = application_train['SK_ID_CURR']

test_binned = binning_process.transform(application_test, metric_missing=0.05)
test_binned.columns = [f'{col}_BIN' for col in test_binned.columns]
test_binned['SK_ID_CURR'] = application_test['SK_ID_CURR']

# Merge bin column to original dataframe
application_train = application_train.merge(train_binned, on = 'SK_ID_CURR', how = 'left')
application_test = application_test.merge(test_binned, on = 'SK_ID_CURR', how = 'left')

application_train['TARGET'] = target_var

In [11]:
print('Training Features shape: ', application_train.shape)
print('Testing Features shape: ', application_test.shape)

Training Features shape:  (246009, 291)
Testing Features shape:  (61502, 290)


In [12]:
application_train = encode(application_train)
application_test = encode(application_test)

In [13]:
print('Training Features shape: ', application_train.shape)
print('Testing Features shape: ', application_test.shape)

Training Features shape:  (246009, 411)
Testing Features shape:  (61502, 410)


In [14]:
## Aligning train and test data
train_labels = application_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
application_train, application_test = application_train.align(application_test, join = 'inner', axis = 1)

# Add the target back in
application_train['TARGET'] = train_labels

print('Training Features shape: ', application_train.shape)
print('Testing Features shape: ', application_test.shape)

Training Features shape:  (246009, 411)
Testing Features shape:  (61502, 410)


In [15]:
application_train.to_csv('application_train_final.csv', index = False)
print('done')

done


In [16]:
application_test.to_csv('application_test_final.csv', index = False)
print('done')

done


### Test on Application_train table

In [17]:
#removing the SK_ID_CURR from training and test data
application_train = application_train.drop(['SK_ID_CURR'], axis = 1)
skid_test = application_test.pop('SK_ID_CURR')


In [18]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(C = 0.1)
train, test = train_test_split(application_train,test_size=.25,random_state = 123)

#separating dependent and independent variables
train_x1 = train[[i for i in train.columns if i not in ['SK_ID_CURR'] + [ 'TARGET']]]
train_y1 = train[["TARGET"]]

test_x1 = test[[i for i in test.columns if i not in ['SK_ID_CURR'] + [ 'TARGET']]]
test_y1 = test[["TARGET"]]

scaler = StandardScaler()
train_x1 = scaler.fit_transform(train_x1)
test_x1 = scaler.fit_transform(test_x1)

In [19]:
log_reg.fit(train_x1, train_y1)
log_reg_pred = log_reg.predict_proba(test_x1)[:, 1]

In [20]:
from sklearn.metrics import roc_auc_score
auc_roc = roc_auc_score(test_y1,log_reg_pred)
gini = 2 * auc_roc - 1
print(f"AUC-ROC Score: {auc_roc}")
print(f"Gini Score: {gini}")

AUC-ROC Score: 0.7590378426832087
Gini Score: 0.5180756853664175
